# The CNS North Korea Missile Test Database

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_grid as altgrid
import numpy as np
import us
import urllib.request, json
import glob
import os
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import os
import time

In [3]:
alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

In [9]:
my_headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko)Chrome/71.0.3578.98 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
}

In [20]:
missiles_url = "https://missilethreat.csis.org/country/dprk"
page = requests.get(missiles_url, headers=my_headers).content

In [29]:
df_list = pd.read_html(page)[0].rename(
    columns={
        "Missile Name": "name",
        "Class": "class",
        "Range": "range",
        "Status": "status",
    }
)

In [33]:
df_list["range"].str.split(" - ", expand=True)

,0,1
0,"2,500","4,000 km"
1,"4,500 km",None
2,"5,500","11,500 km"
3,"10,400 km",None
4,"8,500","13,000 km"
5,300 km,None
6,500 km,None
7,"1,200","1,500 km"
8,800,"1,000 km"
9,110,160 km


### Download latest version of the database from [here](https://beyondparallel.csis.org/database-north-korean-provocations/)

In [ ]:
url = "https://www.nti.org/wp-content/uploads/2021/10/north_korea_missile_test_database_current-for-website.xlsx"

In [ ]:
resp = requests.get(url)
output = open("data/raw/north_korea_missile_test_database.xls", "wb")
output.write(resp.content)
output.close()

#### Read data

In [ ]:
src = pd.read_excel(
    "data/raw/north_korea_missile_test_database.xls", skiprows=1, parse_dates=["Date"]
).sort_values("Date", ascending=False)[
    [
        "Date",
        "Date Entered/Updated",
        "Launch Time (UTC)",
        "Missile Name",
        "Missile Type",
        "Facility Name",
        "Facility Location",
        "Facility Latitude",
        "Facility Longitude",
        "Landing Location",
        "Apogee",
        "Distance Travelled",
        "Confirmation Status",
        "Test Outcome",
        "Additional Information",
        "Source(s)",
    ]
]

In [ ]:
src.columns

In [ ]:
src.columns = (
    src.columns.str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("/", "_", regex=False)
)

#### Dates

In [ ]:
src["year"] = src["date"].dt.year.astype(str)
src["month"] = src["date"].dt.month
src["day"] = src["date"].dt.day
src["day"] = src["date"].dt.day_name()

In [ ]:
type_lookup = {
    "SRBM": "Short-range",
    "MRBM": "Medium-range",
    "SLV": "Satellite launch vehicle",
    "SLBM": "Submarine-lunched",
    "Unknown": "Unknown",
    "IRBM": "Intermediate-range",
    "ICBM": "Intercontinental",
    "HGV": "Hypersonic glide vehicle",
}

In [ ]:
src["missle_type_desc"] = src["missile_type"].map(type_lookup)

In [ ]:
df = src.copy()

In [ ]:
df.head()

---

In [ ]:
missile_types = (
    df.groupby(["year", "missile_type", "missle_type_desc"])["date"]
    .count()
    .reset_index(name="count")
)

In [ ]:
missile_types

In [ ]:
missile_types["missile_type"].value_counts()

---

In [ ]:
alt.Chart(missile_types).mark_circle(
    opacity=0.7, stroke="#1a1a1a", strokeWidth=0.5
).encode(
    alt.X(
        "year:O",
        axis=alt.Axis(
            tickCount=2,
        ),
        title="",
    ),
    alt.Y("missle_type_desc:N", title=""),
    alt.Size(
        "count:Q",
        scale=alt.Scale(range=[1, 2000]),
        legend=None,
    ),
    alt.Color("missile_type:N", legend=None),
).properties(
    width=650, height=250, title="North Korea missile launches by type and year"
).configure_legend(
    orient="top", symbolType="circle"
)

---